In [ ]:
import os
import cv2
import PIL.Image as Image
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from enhanced_model import enhance_net

In [ ]:
def NormalizeData(data):
    return (data - tf.math.reduce_min(data)) / (tf.math.reduce_max(data) - tf.math.reduce_min(data))

In [ ]:
def image_selection(enhancement_images):
    images = []
    select_img = []
    select_img_safe = [[],[],[],[],[],[],[],[]]
    for index, i in enumerate(enhancement_images):
        r,g,b = tf.split(i, 3, axis=2)
        r_g = int(np.abs(np.mean(r*0.299-g*0.587)))
        r_b = int(np.abs(np.mean(r*0.299-b*0.114)))
        g_b = int(np.abs(np.mean(g*0.587-b*0.114)))
        
        img = tf.image.rgb_to_hsv(i)
        h,s,v = tf.split(img, 3, axis=2)
        mean_hue = tf.math.reduce_mean(h*255)
        mean_v = tf.math.reduce_mean(v)
        

        if 22 < mean_hue and mean_hue <= 100:
            select_img_safe[index].append('hue')
            if 30 <= mean_v and mean_v <= 145:
                select_img_safe[index].append('lightness')
                if  5<=r_g and  r_g <= 36: 
                    select_img_safe[index].append('r-g')
                    if 5 <= r_b and r_b<=48:
                        select_img_safe[index].append('r-b')
                        if 5 <= g_b and  g_b <= 75:
                            select_img_safe[index].append('g-b')
                            images.append(i)
                            select_img.append(index)
    if np.shape(images)[0] == 0:
        print(select_img_safe)
        return enhancement_images[0:5]
    print(select_img)
    return images

In [ ]:
model_name = ['DCE', 'CSP_DCE', 'MSP_DCE','DCE++']
model_name = model_name[1]

print(model_name)

In [ ]:
i = 10
path = './model/{0}/weights/epoch{1}/'.format(model_name, i)
# path = './model/DCE_1_10_5_1600/weights/epoch{0}/'.format(i)
model = enhance_net(input_shape=(None,None,3), model_name=model_name)
model.enhancement_net.load_weights(path)    

In [ ]:
img_dir = './Dataset/denoise/LOL/train/low/'
# img_dir = './result/paper_use/Visual Comparisons/low/output/'
# img_dir = 'C://Users/user/Downloads/mh/'
output_dir = './result/{0}/epoch{1}/'.format(model_name, i)

folder = os.path.exists(output_dir)
if not folder:
    os.makedirs(output_dir)
for i in range(5):
    strat = time.time()
    for file in os.listdir(img_dir):
        img_path = img_dir + file
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
        
        h,w,c = np.shape(img)
        img = tf.reshape(img, (1,h,w,c))
        img = tf.cast(img, dtype=tf.float32)
        
        enhancemnet_image, parameter_map = model.predict(img)
        
        enhancemnet_image = NormalizeData(enhancemnet_image)
        enhancemnet_image = np.reshape(enhancemnet_image, (h,w,c))
        enhancemnet_image = cv2.cvtColor(enhancemnet_image, cv2.COLOR_RGB2BGR)
        # cv2.imwrite(output_dir +file, enhancemnet_image*255)

    end = time.time()
    per_image_time = (end - strat)/len(os.listdir(img_dir))
    print(per_image_time)
        

In [ ]:
# i = 2
for i in range(10,101,10):
    path = './model/{0}/weights/epoch{1}/'.format(model_name, i)
    # path = './model/DCE_1_10_5_1600/weights/epoch{0}/'.format(i)
    model = enhance_net(input_shape=(None,None,3), model_name=model_name)
    model.enhancement_net.load_weights(path)    
    
    # img_dir = './Dataset/denoise/LOL/train/low/'
    img_dir = './Dataset/denoise/test/low/'
    # img_dir = 'C://Users/user/Downloads/mh/'
    output_dir = './result/{0}/epoch{1}/'.format(model_name, i)

    folder = os.path.exists(output_dir)
    if not folder:
        os.makedirs(output_dir)

    strat = time.time()
    for file in os.listdir(img_dir):
        img_path = img_dir + file
        img = cv2.imread(img_path)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
        h,w,c = np.shape(img)
        img = tf.reshape(img, (1,h,w,c))
        pred_img = tf.cast(img, dtype=tf.float32)
        
        
        enhancemnet_image, parameter_map = model.predict(pred_img)
    
        enhancemnet_image = NormalizeData(enhancemnet_image)
        enhancemnet_image = np.reshape(enhancemnet_image, (h,w,c))
        enhancemnet_image = cv2.cvtColor(enhancemnet_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_dir +file, enhancemnet_image*255)
    
    
        # ----------------對low-light Image增強----------------
        # enhancement_images = []
        # for i in range(8):
        #     pred_img = pred_img + parameter_map * (pred_img - tf.math.pow(pred_img, 2))
        #     # pred_img = NormalizeData(pred_img)
        #     enhancement_image = np.reshape(pred_img, (h,w,c))*255
        #     cv2.imwrite("./result/test/{0}_{1}".format(i+1,file), cv2.cvtColor(np.asarray(enhancement_image), cv2.COLOR_RGB2BGR))
        #     enhancement_images.append(enhancement_image)
        # -----------------------end--------------------------
    
    
        # # ------------------------
        # print(file, end=' ')
        # print(np.shape(enhancement_images), end = '')
        # enhancement_images = image_selection(enhancement_images)
        # # ------------------------
        
        
        # # ------------Merge using Exposure Fusion-------------
        # # print("Merging using Exposure Fusion ... ")
        # mergeMertens = cv2.createMergeMertens()
        # exposureFusion = mergeMertens.process(enhancement_images[-2:])
        # # plt.imshow(exposureFusion)
        # # plt.show()

        # # Save output image
        # # print("Saving output ... exposure-fusion.jpg")
        # exposureFusion = cv2.cvtColor(exposureFusion,cv2.COLOR_BGR2RGB)
        # cv2.imwrite(output_dir + file[:-4]+'exposure-fusion.png', exposureFusion*255)
        # # -----------------------end--------------------------
        
    end = time.time()
    per_image_time = (end - strat)/len(os.listdir(img_dir))
    print(per_image_time)

In [ ]:
import os
import cv2
import PIL.Image as Image
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from model import enhance_net

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

model_name = ['DCE', 'CSP_DCE','MSP_DCE', 'DCE++']
model_name = model_name[1]

print(model_name)

i = 10
path = './model/{0}/weights/epoch{1}/'.format('MSP_DCE', i)
model = enhance_net(input_shape=(None,None,3), model_name=model_name)
model.enhancement_net.load_weights(path) 

img_path = './result/paper_use/Visual Quality Evaluation/low/778.png'
img = cv2.imread(img_path)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)/255
h,w,c = np.shape(img)
img = tf.reshape(img, (1,h,w,c))
pred_img = tf.cast(img, dtype=tf.float32)

enhancement_image, parameter_map = model.predict(pred_img)

In [ ]:
enhancement_image = np.reshape(enhancement_image, (h,w,c))
enhancement_image = cv2.cvtColor(enhancement_image, cv2.COLOR_RGB2BGR)
enhancement_image = NormalizeData(enhancement_image)
cv2.imwrite('enhanced_img.png', enhancement_image*255)

In [ ]:
parameter_map = np.reshape(parameter_map, (h,w,c))
parameter_map = cv2.cvtColor(parameter_map, cv2.COLOR_RGB2BGR)
p_r, p_g, p_b = cv2.split(parameter_map)
p_r = NormalizeData(p_r)
p_g = NormalizeData(p_g)
p_b = NormalizeData(p_b)
cv2.imwrite('parameter_map_R.png', p_r*255)
cv2.imwrite('parameter_map_G.png', p_g*255)
cv2.imwrite('parameter_map_B.png', p_b*255)

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(1, 3, 1)
plt.imshow(p_r, cmap='gray')
plt.subplot(1, 3, 2)
plt.imshow(p_g, cmap='gray')
plt.subplot(1, 3, 3)
plt.imshow(p_b, cmap='gray')
plt.show()

In [ ]:
A13,A24 = np.split(parameter_map,2,axis=0)

In [ ]:
A1,A3 = np.split(A13,2,axis=1)
A2,A4 = np.split(A24,2,axis=1)

cv2.imwrite('parameter_map.png', parameter_map*255)
cv2.imwrite('A1.png', A1*255)
cv2.imwrite('A2.png', A2*255)
cv2.imwrite('A3.png', A3*255)
cv2.imwrite('A4.png', A4*255)

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(1, 4, 1)
plt.imshow(A1, cmap='gray')
plt.subplot(1, 4, 2)
plt.imshow(A2, cmap='gray')
plt.subplot(1, 4, 3)
plt.imshow(A3, cmap='gray')
plt.subplot(1, 4, 4)
plt.imshow(A4, cmap='gray')
plt.show()